In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, roc_curve, auc, precision_recall_curve
from sklearn.model_selection import train_test_split

In [4]:
preprocessed_data = pd.read_csv(r'/Users/issackondreddy/Desktop/Projects/Fraud Detection System/DS/data/processed_creditcard.csv')
preprocessed_data.head()

,Scaled_Time,Scaled_Amount,V1,V2,V3,V4,V5,V6,V7,V8,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
0,-1.996583,0.244964,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0
1,-1.996583,-0.342475,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0
2,-1.996562,0.140534,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0
3,-1.996541,-0.073403,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0
4,-1.996541,-0.338556,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,...,0.084968,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,0


In [5]:
# Separate features and target
X = preprocessed_data.drop('Class', axis=1)
y = preprocessed_data['Class']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Reshape data for RNN (Add a time-step dimension for LSTM/GRU input)
X_train_rnn = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)  # Reshape for RNN (LSTM/GRU)
X_test_rnn = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

In [7]:
model = models.Sequential()

# Input layer and LSTM layer with 128 units (to match the progression of Conv1D filters)
model.add(layers.LSTM(units=128, return_sequences=True, input_shape=(X_train_rnn.shape[1], 1)))
model.add(layers.LSTM(units=64, return_sequences=True))  # Matching the next Conv1D layer with 64 filters
model.add(layers.LSTM(units=32))  # Matching the final Conv1D layer with 32 filters

# Flatten and add dense layers
model.add(layers.Dense(100, activation='relu'))  # Matching the first dense layer in the CNN
model.add(layers.Dense(50, activation='relu'))   # Matching the second dense layer in the CNN
model.add(layers.Dense(1, activation='sigmoid'))  # Final output layer for binary classification

/Users/issackondreddy/Desktop/Projects/Fraud Detection System/FDS/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the RNN model
history = model.fit(X_train_rnn, y_train, epochs=10, batch_size=32, validation_data=(X_test_rnn, y_test))

Epoch 1/10
 839/7565 ━━━━━━━━━━━━━━━━━━━━ 6:51 61ms/step - accuracy: 0.8983 - loss: 0.2916

In [ ]:
test_loss, test_acc = model.evaluate(X_test_rnn, y_test)
print(f"Test Accuracy: {test_acc}")


In [ ]:
# Get model predictions and compute relevant metrics
y_pred = (model.predict(X_test_rnn) > 0.5).astype("int32")

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
# Plot Accuracy and Loss Curves
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy over epochs')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss over epochs')

plt.show()

In [ ]:


# Compute and plot ROC Curve
fpr, tpr, _ = roc_curve(y_test, model.predict(X_test_rnn))
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:0.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
precision, recall, _ = precision_recall_curve(y_test, model.predict(X_test_rnn))

plt.figure(figsize=(6, 6))
plt.plot(recall, precision, lw=2, color='blue')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.show()